In [3]:
import pandas as pd
from pathlib import Path

In [4]:
eval_path = Path("../../data/evaluation/")
load_eval_queries = pd.read_csv(eval_path / "LLM Evaluation - Topic Queries.csv")[
    ["Topic", "Dataset IDs"]
]

In [5]:
# load_eval_queries = load_eval_queries.loc[50:]
# load_eval_queries

In [6]:
from collections import defaultdict

dict_ids = defaultdict(list)

for i, row in load_eval_queries.iterrows():
    dids = row["Dataset IDs"].split(",")
    for did in dids:
        dict_ids[did].append(row["Topic"])
dict_ids = dict(dict_ids)

In [1]:
import pandas as pd
from pathlib import Path
import glob
from tqdm.auto import tqdm
import json

/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
eval_path = Path("../../data/evaluation/")

In [3]:
# load_eval_queries = pd.read_csv(eval_path / "LLM Evaluation - Topic Queries.csv")[["Topic", "Dataset IDs"]]
load_eval_queries = pd.read_csv(eval_path / "merged_labels.csv")[
    ["Topics", "Dataset IDs"]
]

In [4]:
# read templates and add information to the queries
with open(eval_path / "query_templates.txt", "r") as f:
    query_templates = f.readlines()
    query_templates = [x.strip() for x in query_templates]

In [5]:
query_key_dict = {}
for template in query_templates:
    for row in load_eval_queries.itertuples():
        new_query = f"{template} {row[1]}".strip()
        # load_eval_queries.at[query, "Query"] = new_query
        if new_query not in query_key_dict:
            query_key_dict[new_query.strip()] = row[2]

In [15]:
def create_results_dict(csv_files, query_key_dict):
    merged_df = pd.DataFrame()

    for exp_path in tqdm(csv_files):

        exp = pd.read_csv(exp_path).rename(columns={"did": "y_pred"})
        exp["llm_before_rag"] = exp["llm_before_rag"].fillna("None")
        exp["y_pred"] = exp["y_pred"].astype(str)
        exp["query"] = exp["query"].str.strip()

        exp["y_true"] = exp["query"].map(query_key_dict)

        exp["y_true"] = exp["y_true"].str.split(",")

        exp["exists_in"] = [
            any(x in y for x in y_pred)
            for y_pred, y in zip(exp["y_pred"], exp["y_true"])
        ]

        merged_df = pd.concat([merged_df, exp])

    return merged_df

In [16]:
# glob all csv files in the experiments directory
csv_files = glob.glob(str(eval_path / "*/*/results.csv"))
csv_files

['../../data/evaluation/Snowflake_snowflake-arctic-embed-l/Snowflake_snowflake-arctic-embed-l_phi3/results.csv',
 '../../data/evaluation/Snowflake_snowflake-arctic-embed-l/Snowflake_snowflake-arctic-embed-l_llama3/results.csv',
 '../../data/evaluation/BAAI_bge-large-en-v1.5/BAAI_bge-large-en-v1.5_phi3/results.csv',
 '../../data/evaluation/BAAI_bge-large-en-v1.5/BAAI_bge-large-en-v1.5_llama3/results.csv',
 '../../data/evaluation/BAAI_bge-base-en-v1.5/BAAI_bge-base-en-v1.5_phi3/results.csv',
 '../../data/evaluation/BAAI_bge-base-en-v1.5/BAAI_bge-base-en-v1.5_llama3/results.csv']

In [17]:
results_df = create_results_dict(csv_files, query_key_dict=query_key_dict)

 17%|█▋        | 1/6 [00:00<00:00,  6.07it/s]

../../data/evaluation/Snowflake_snowflake-arctic-embed-l/Snowflake_snowflake-arctic-embed-l_phi3/results.csv
../../data/evaluation/Snowflake_snowflake-arctic-embed-l/Snowflake_snowflake-arctic-embed-l_llama3/results.csv


 50%|█████     | 3/6 [00:00<00:00,  5.69it/s]

../../data/evaluation/BAAI_bge-large-en-v1.5/BAAI_bge-large-en-v1.5_phi3/results.csv
../../data/evaluation/BAAI_bge-large-en-v1.5/BAAI_bge-large-en-v1.5_llama3/results.csv


 83%|████████▎ | 5/6 [00:00<00:00,  5.73it/s]

../../data/evaluation/BAAI_bge-base-en-v1.5/BAAI_bge-base-en-v1.5_phi3/results.csv


100%|██████████| 6/6 [00:00<00:00,  6.08it/s]

../../data/evaluation/BAAI_bge-base-en-v1.5/BAAI_bge-base-en-v1.5_llama3/results.csv


In [18]:
results_df["llm_before_rag"].value_counts()

llm_before_rag
None     276120
True      92392
False     92392
Name: count, dtype: int64

In [20]:
grouped_value_counts = results_df.groupby(
    ["embedding_model", "llm_model", "llm_before_rag"]
)["exists_in"].value_counts()

In [21]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
pd.DataFrame(grouped_value_counts)

count
embedding_model                    llm_model llm_before_rag exists_in       
BAAI/bge-base-en-v1.5              llama3    False          False      14008
                                                            True        1168
                                             True           False      14008
                                                            True        1168
                                             None           False      43104
                                                            True        2916
                                   phi3      False          False      14008
                                                            True        1168
                                             True           False      14008
                                                            True        1168
                                             None           False      43104
                                                            True        2916
BAAI/bge-large-en-v1.5             llama3    False          False      13800
                                                            True        1171
                                             True           False      13800
                                                            True        1171
                                             None           False      43104
                                                            True        2916
                                   phi3      False          False      13800
                                                            True        1171
                                             True           False      13800
                                                            True        1171
                                             None           False      43104
                                                            True        2916
Snowflake/snowflake-arctic-embed-l llama3    False          False      14802
                                                            True        1247
                                             True           False      14802
                                                            True        1247
                                             None           False      43104
                                                            True        2916
                                   phi3      False          False      14802
                                                            True        1247
                                             True           False      14802
                                                            True        1247
                                             None           False      43104
                                                            True        2916